In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#class representing one simulation.
#an instance of this class contains enough information to plot some pictures.
#the return value for sim_class and for recurrent_sim_class is an instance of this class. 

class Sim:
    def __init__(self, outcome, times, betas, gammas, N_infecteds, deltas=None, spillover_ids=None, S=None, D=None):
        
        #0 if extinction, 1 if outbreak, None otherwise. 
        #(note sim_class will always return 0 or 1 here, but recurrent_sim_class may return None)
        self.outcome = outcome 
        
        #the number of events in the simulation
        self.length = len(times) # note this is = len(self.betas) = len(self.gammas)= len(self.N)
        
        #these are all arrays of the same length
        self.times = times #all the times of new events (infections and recoveries, including spillover infections)
        self.betas = betas #the corresponding betas. if the event was a recovery, the corresponding beta is "None"
        self.gammas = gammas #the corresponding gammas. if the event was a recovery, the corresponding gamma is "None"
        self.N = N_infecteds #the number of infecteds, inclusive of the new event
        self.spillover_ids = spillover_ids #which spillover tree each event belongs to. the spillover trees ids are integers beginning with 0
        self.deltas = deltas
        self.S = S
        self.D = D
    
        if spillover_ids is None: #if there are no ids, assume they are all part of the same tree, so make them all 0.
            self.spillover_ids = np.zeros(self.length)
            
            
    
    #returns the ith event [t, beta, gamma, N]
    def event(self, i):
        return np.array([self.times[i], self.betas[i], self.gammas[i], self.N[i]])
    
    #truncate everything after time t_thresh
    def truncate(self, t_thresh):      
        for i in range(self.length()):
            if self.times[i] > t_thresh:
                self.times = self.times[:i]
                self.betas = self.betas[:i]
                self.gammas = self.gammas[:i]
                self.N = self.N[:i]
                break
        

In [3]:
################################################
#
# version of recurrent_sim which runs on a finite population in an SIS fashion, distinguishes between recovery and death, and returns an element of the class Sim.
#
# will run until either we get an outbreak, or the simulation times out at specified threshold time.
# there is one other option, which occurs only if alpha = 0 and the infection goes extinct.
# (when alpha > 0, we will keep running, instead of stopping, at an extinction time.)
#
################################################
#
# input S_0 is size of initial susceptible population
#
# optional input t_thresh is the maximum number of simulated days to run.
# optional input outbreak_thresh. we call it an outbreak if this many people are infected 
#
# returns an instance of the class Sim.
# the return object will be "None" for self.outcome if the simulation timed out by hitting the threshold time. 
# otherwise if we had an outbreak, self.outcome will be 1.
# finally, if alpha=0 and we had extinction, self.outcome will be 0.
#
#################################################

def recurrent_sim_death_class_finite(S_0, alpha, beta_0, gamma_0, delta_0, mu_1, mu_2, mu_3, t_thresh = 1000, outbreak_thresh = 100):
    
    t=0
        
    #stuff to fill and return as part of Sim object.
    times = np.array([t])
    all_betas = np.array([beta_0]); all_gammas = np.array([gamma_0]); all_deltas = np.array([delta_0])
    all_N = np.array([1]); all_S = np.array([S_0]); all_D = np.array([0])
    spillover_ids = np.array([0]) #keep track of which spillover tree each event belongs to
    
    #keep track of how many spillover trees we have seen so far
    spillovers = 0
    
    #initialize variables to keep track of number of susceptible, infected, deads
    S = S_0; N = 1; D = 0 
        
    #initialize variables to keep track of sums of beta, gamma, and delta over all currently infected people    
    beta_sum = beta_0; gamma_sum = gamma_0; delta_sum = delta_0
        
    #initialize matrix of active cases. each case is a row of length 5.
    infecteds=np.array([[beta_0, gamma_0, delta_0, t, spillovers]])
    
    while True:
                       
        total_pop = S + N #total population (this is what's usually called N.)
        
        #grab arrays of all betas gammas deltas
        betas = infecteds[:,0]; gammas = infecteds[:,1]; deltas = infecteds[:,2]
        
        #adjust betas to actual transmission rates, which depend on proportion of population that is susceptible
        trans_rates = betas*S/total_pop
        
        trans_rates_sum = np.sum(trans_rates) #sum all trans rates               
                       
        #compute interevent time (for the whole population)

        #rate of events is alpha + (sum of transmission rate and gamma and delta) over all infected people
        overall_rate = trans_rates_sum + gamma_sum + delta_sum + alpha  
                
        #if alpha = 0 and we had extinction
        if alpha == 0 and N == 0:
            #print("extinction")
            return Sim(0, times, all_betas, all_gammas, all_N, all_deltas, spillover_ids, all_S, all_D)               
                       
        #draw from exponential distribution with this rate
        dt = np.random.exponential(scale=1/overall_rate)
        t += dt            
        
        #check if we exceeded t_thresh
        if t > t_thresh:
            #print('exceeded t_thresh')
            return Sim(None, times, all_betas, all_gammas, all_N, all_deltas, spillover_ids, all_S, all_D)
        
        #figure out whether the event was a new spillover
        event = np.random.rand() #draw from uniform distribution over [0, 1)
        p_spillover = alpha/overall_rate
            
        if event < p_spillover: #new spillover
            
            #update total spillovers counter
            spillovers += 1
            
            #append new case to infecteds array
            infecteds = np.append(infecteds, [[beta_0, gamma_0, delta_0, t, spillovers]], axis=0)

            #update the sums of beta gamma delta
            beta_sum += beta_0; gamma_sum += gamma_0; delta_sum += delta_0

            #update N
            N += 1
            print("N=",N)
            
            #update S
            S -= 1
            print("S=",S)

            #update return arrays
            times = np.append(times, t)
            all_betas = np.append(all_betas, beta_0); all_gammas = np.append(all_gammas, gamma_0); all_deltas = np.append(all_deltas, delta_0)
            all_N = np.append(all_N, N); all_S = np.append(all_S, S); all_D = np.append(all_D, D)
            spillover_ids = np.append(spillover_ids, spillovers)

#             #check if we have an outbreak
#             if N >= outbreak_thresh:
#                 #print("outbreak!")
#                 return Sim(1, times, all_betas, all_gammas, all_N, spillover_ids)

            continue #skip to next event
        
        #otherwise, the event was that somebody either transmitted or recovered
                       
                       
        #figure out who the event happened to
        i = np.random.choice(N, p=(trans_rates+gammas+deltas)/(overall_rate-alpha)) #index of that person
        
        #grab their specific transmission rate and beta and gamma and delta and spillover id
        beta = betas[i]; gamma = gammas[i]; delta = deltas[i]; spillover_id = infecteds[i][4]
        
        #compute transmission rate
        trans_rate = beta*S/total_pop
        
        #figure out what they did, transmit, recover, or die
        ev = np.random.rand() #draw from uniform distribution over [0, 1)
        prob_trans = trans_rate / (trans_rate + gamma + delta) #probability of transmission
        prob_recovery = gamma / (trans_rate + gamma + delta) #probability of recovery                      
                       
        if (ev < prob_trans): #transmission

            #pick beta, gamma, and delta for new case
            #mutation is a number drawn from normal distribution with std dev mu_1 or mu_2 or mu_3
            #don't allow negative beta.
            #don't allow gamma to be negative. 
            #don't allow delta to be less than a small value, the natural death rate.
            mut1 = np.random.normal(loc=0.0, scale=mu_1)
            new_beta = max(0, beta + mut1)
            mut2 = np.random.normal(loc=0.0, scale=mu_2)
            new_gamma = max(0, gamma + mut2)
            mut3 = np.random.normal(loc=0.0, scale=mu_3)              
            nat_death = 0.00002366575 #taken from CDC: 
                #https://www.cdc.gov/nchs/fastats/deaths.htm & scaled to be daily rate instead of yearly
            new_delta = max(nat_death, delta + mut3)

            #append new case to infecteds array
            infecteds = np.append(infecteds, [[new_beta, new_gamma, new_delta, t, spillover_id]], axis=0)
            
            #update the sums of beta, gamma and delta
            beta_sum += new_beta; gamma_sum += new_gamma; delta_sum += new_delta
            
            #update N
            N += 1
            
            #update S
            S -= 1
            
            #update return arrays
            times = np.append(times, t)
            all_betas = np.append(all_betas, new_beta)
            all_gammas = np.append(all_gammas, new_gamma)
            all_deltas = np.append(all_deltas, new_delta)
            all_N = np.append(all_N, N) 
            spillover_ids = np.append(spillover_ids, spillover_id)
            all_S = np.append(all_S, S)
            all_D = np.append(all_D, D)

#             #check if we have an outbreak
#             if N >= outbreak_thresh:
#                 #print("\n", "outbreak!")
#                 return Sim(1, times, all_betas, all_gammas, all_N, all_deltas, spillover_ids)
        
        elif (ev < prob_trans + prob_recovery): #recovery
            
            #delete them from infecteds array
            infecteds = np.delete(infecteds, i, axis=0)
            
            #update the sums of beta, gamma and delta
            beta_sum += beta; gamma_sum -= gamma; delta_sum -= delta
            
            #update N
            N -= 1
            
            #update S
            S += 1
            
            #update return arrays
            times = np.append(times, t)
            all_betas = np.append(all_betas, None)
            all_gammas = np.append(all_gammas, None)
            all_deltas = np.append(all_deltas, None)
            all_N = np.append(all_N, N)
            spillover_ids = np.append(spillover_ids, spillover_id) 
            all_S = np.append(all_S, S)
            all_D = np.append(all_D, D)
        
        else: #death
            
            #delete them from infecteds array
            infecteds = np.delete(infecteds, i, axis=0)
            
            #update the sums of beta, gamma and delta
            beta_sum += beta; gamma_sum -= gamma; delta_sum -= delta
            
            #update N
            N -= 1
            
            #update D
            D += 1
            
            #update return arrays
            times = np.append(times, t)
            all_betas = np.append(all_betas, None)
            all_gammas = np.append(all_gammas, None)
            all_deltas = np.append(all_deltas, None)
            all_N = np.append(all_N, N)
            spillover_ids = np.append(spillover_ids, spillover_id)
            all_S = np.append(all_S, S)
            all_D = np.append(all_D, D)

In [4]:
#color cycles for different spillover trees
def colors(ids):
    dict = {0: 'red', 1: 'orange', 2: 'yellow', 3: 'green', 4: 'blue', 5: 'purple' }
    colors = []
    for id in ids:  
        colors.append(dict[id%6])
    return colors

#parameters
S_0 = 1000
beta_0 = 0.1
gamma_0 = 0.05
delta_0 = 0.05
mu_1 = 0.1
mu_2 = 0.01
mu_3 = 0.01
alpha = 0.05

t_thresh = 1000
outbreak_thresh = 100

#compute the simulation

#s = recurrent_sim_death_class_finite(S_0, alpha, beta_0, gamma_0, delta_0, mu_1, mu_2, mu_3, t_thresh, outbreak_thresh)

s = None
while True:
    s = recurrent_sim_death_class_finite(S_0, alpha, beta_0, gamma_0, delta_0, mu_1, mu_2, mu_3, t_thresh, outbreak_thresh)
    if s.times[-1] > 500: break
    #if s.outcome == 1: break
        
times = s.times; betas = s.betas; gammas = s.gammas; deltas = s.deltas
N_infecteds = s.N; S = s.S; D = s.D
spillover_ids = s.spillover_ids

print('outcome = ', s.outcome)

#plots

############################################ OPTION 1 #########################################################
############# plot each parameter separately, distinguishing between different spillover trees ################

# #plot betas over time
# plt.figure(figsize=(20,6))
# plt.scatter(times, betas, marker = ".", c=colors(spillover_ids), label = 'beta')
# plt.title('alpha = {}, mu_1 = {}, mu_2 = {}, mu_3 = {}'.format(alpha, mu_1, mu_2, mu_3))
# plt.xlabel("t"); plt.ylabel("parameters")
# plt.autoscale(enable=True, axis='x', tight=True)
# plt.legend(); plt.show()

# #plot gammas over time
# plt.figure(figsize=(20,6))
# plt.scatter(times, gammas, marker = ".", c=colors(spillover_ids), label = 'gamma')
# plt.xlabel("t"); plt.ylabel("gamma")
# plt.autoscale(enable=True, axis='x', tight=True)
# plt.show()

# #plot deltas over time
# plt.figure(figsize=(20,6))
# plt.scatter(times, deltas, marker = ".", c=colors(spillover_ids), label = 'delta')
# plt.xlabel("t"); plt.ylabel("delta")
# plt.autoscale(enable=True, axis='x', tight=True)
# plt.show()


############################################ OPTION 2 #########################################################
######################## plot beta separately, and other 2 parameters together ################################
########################### without distinguishing between spillover trees ####################################

#plot betas over time
plt.figure(figsize=(20,6))
plt.plot(times, betas, ".r", label='beta')
plt.title('mu_1 = {}, mu_2 = {}, mu_3 = {}'.format(mu_1, mu_2, mu_3))
plt.xlabel("t"); plt.ylabel("parameters")
plt.autoscale(enable=True, axis='x', tight=True)
plt.legend(); plt.show()

#plot gammas and deltas over time
plt.figure(figsize=(20,6))
plt.plot(times, gammas, ".g", label='gamma')
plt.plot(times, deltas, ".b", label='delta')
plt.xlabel("t"); plt.ylabel("parameters")
plt.autoscale(enable=True, axis='x', tight=True)
plt.legend(); plt.show()


############################################ OPTION 3 #########################################################
################################ plot all 3 parameters together ###############################################
########################### without distinguishing between spillover trees ####################################

# #plot parameters over time
# plt.figure(figsize=(20,6))
# plt.plot(times, betas, ".r", label='beta')
# plt.plot(times, gammas, ".g", label='gamma')
# plt.plot(times, deltas, ".b", label='delta')
# plt.title('alpha = {}, mu_1 = {}, mu_2 = {}, mu_3 = {}'.format(alpha, mu_1, mu_2, mu_3))
# plt.xlabel("t"); plt.ylabel("parameters")
# plt.autoscale(enable=True, axis='x', tight=True)
# plt.legend(); plt.show()

###############################################################################################################

#plot infecteds, susceptible, dead over time
plt.figure(figsize=(20,6))
plt.plot(times, N_infecteds, label='infected')
plt.plot(times, S, label='susceptible')
plt.plot(times, D, label='dead')
plt.xlabel("t"); plt.ylabel("\n number of people \n")
plt.autoscale(enable=True, axis='x', tight=True)
#plt.ylim(0, outbreak_thresh)
plt.legend(); plt.show()

N= 2
S= 999
N= 1
S= 1000
N= 3
S= 998
N= 3
S= 998
N= 4
S= 994
N= 3
S= 993
N= 4
S= 991
N= 13
S= 976
N= 24
S= 963
N= 721
S= 130
N= 179
S= 1
N= 110
S= 0
N= 102
S= -1


ValueError: scale < 0